In [ ]:
import tensorflow as tf

In [ ]:
class switch(object):
    """Switch statement for Python, based on recipe from Python Cookbook."""

    def __init__(self, value):
        self.value = value
        self.fall = False

    def __iter__(self):
        """Return the match method once, then stop"""
        yield self.match
        raise StopIteration
    
    def match(self, *args):
        """Indicate whether or not to enter a case suite"""
        if self.fall or not args:
            return True
        elif self.value in args: # changed for v1.5
            self.fall = True
            return True
        else:
            return False

In [ ]:
# test_dict = {"hi": 5, "hey": 3}
# test_dict = ["hi", "hey"]
test_dict = "hi"

for case in switch(test_dict):
    print(case("hi"))
    if case('dfsd'):
        print(55)
    elif case('hi'):
        print(23423)

In [ ]:
# https://stats.stackexchange.com/questions/330176/what-is-the-output-of-a-tf-nn-dynamic-rnn
tf.reset_default_graph()
n_steps = 2
n_inputs = 3
n_neurons = 5

X = tf.placeholder(dtype=tf.float32, shape=[None, n_steps, n_inputs])
seq_length = tf.placeholder(tf.int32, [None])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, sequence_length=seq_length, dtype=tf.float32)

X_batch = np.array([
  # t = 0      t = 1
  [[0, 1, 2], [9, 8, 7]], # instance 0
  [[3, 4, 5], [0, 0, 0]], # instance 1
  [[6, 7, 8], [6, 5, 4]], # instance 2
  [[9, 0, 1], [3, 2, 1]], # instance 3
])
seq_length_batch = np.array([2, 1, 2, 2])


In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    outputs_val, states_val = sess.run([outputs, states],
                                       feed_dict={
                                           X: X_batch,
                                           seq_length: seq_length_batch
                                       })

    print(outputs_val.shape)
    print()
    print(states_val.shape)

In [ ]:
tf.reset_default_graph()
# x = tf.constant([[1., 1.], [2., 2.]])
x = tf.random.uniform((4,5,6,7))
y = tf.reduce_mean(x,axis=1)
z = tf.transpose(x, perm=[0,1,3,2])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(x.shape)
    print(y.shape)
    print(z.shape)
    print(x)
# x.shape

In [ ]:
computing = {
    'functions_on_devices': {'/cpu:0': ['point_to_coordinate']},
    'default_device': ''
    
}
{k: computing[k] for k in ('functions_on_devices', 'default_device')}

In [ ]:
len("TENLYFQSMGLRTVEMKKGPTDSLGISIAGGVGSPLGDVPIFIAMMHPTGVAAQTQKLRVGDRIVTICGTSTEGMTHTQAVNLLKNASGSIEMQVVAGGDVSETSV")

In [ ]:
len("----------++++++++----++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++--------")

In [ ]:
base_dir = "input_output/"
sum1 = "summary1.txt"
sum2 = "summary2.txt"
casp = "casp12.txt"
outfile = "scatter_plot_data.txt"

In [ ]:
def create_eval_table(file1, file2, casp_map, outfile):
    """Parse two summary files and create a table
    
    Args:
        file1, file2: summary files to parse
        casp_map: a file that maps protein name to FM or TBM in CASP
        outfile: file to write to
    
    Returns:
        None
    """

    def parse_file(file):
        return open(file, "r").read().split('\n')[1:][:-1]

    def out_line(i1, i2, i3, i4, i5, i6):
        line = "{}\t{}\t{}\t{}\t{}\t{}\n".format(i1, i2, i3, i4, i5, i6)

        return line

    f1 = parse_file(file1)
    f2 = parse_file(file2)
    average1 = f1[-1].split('\t')
    average2 = f2[-1].split('\t')
    f1 = f1[:-1]
    f2 = f2[:-1]

    out = open(outfile, "w")
    out.write(
        out_line('TMScore1', 'TMScore2', 'RMSD1', 'RMSD2', 'Name',
                 'Casp_classification'))

    casp_map = parse_file(casp_map)
    casp_dict = dict()
    for line in casp_map:
        row_values = line.split()
        casp_dict['T0' + row_values[0]] = row_values[1]

    for i, line in enumerate(f1):
        row_values1 = line.split('\t')
        row_values2 = f2[i].split('\t')

        line = out_line(row_values1[1], row_values2[1], row_values1[2],
                        row_values2[2], row_values1[0],
                        casp_dict[row_values1[0]])
        out.write(line)

    out.write(
        out_line(average1[1], average2[1], average1[2], average2[2],
                 average1[0], 'None'))


create_eval_table(base_dir + sum1, base_dir + sum2, base_dir + casp,
                  base_dir + outfile)

In [ ]:
import seaborn as sns
# sns.set()
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# ax = sns.scatterplot(x="total_bill", y="tip", data=tips)

In [ ]:
def create_df(file1, file2, casp_map):
    """Parse two summary files and create a table
    
    Args:
        file1, file2: summary files to parse
        casp_map: a file that maps protein name to FM or TBM in CASP
        outfile: file to write to
    
    Returns:
        None
    """

    def parse_file(file):
        return open(file, "r").read().split('\n')[1:][:-1]

    f1 = parse_file(file1)[:-1]
    f2 = parse_file(file2)[:-1]

    tmscores1 = []
    tmscores2 = []
    rmsd1 = []
    rmsd2 = []
    names = []
    casp_classification = []

    casp_map = parse_file(casp_map)
    casp_dict = dict()
    for line in casp_map:
        row_values = line.split()
        casp_dict['T0' + row_values[0]] = row_values[1]

    for i, line in enumerate(f1):
        row_values1 = line.split('\t')
        row_values2 = f2[i].split('\t')
        
        tmscores1.append(float(row_values1[1]))
        tmscores2.append(float(row_values2[1]))
        rmsd1.append(float(row_values1[2]))
        rmsd2.append(float(row_values2[2]))
        names.append(row_values1[0])
        casp_classification.append(casp_dict[row_values1[0]])
        
    df = pd.DataFrame({
        'TMScore1': tmscores1,
        'TMScore2': tmscores2, 
        'RMSD1': rmsd1, 
        'RMSD2': rmsd2, 
        'Name': names,
        'Casp_classification': casp_classification
    })
    
    df['TMScore1'] = df['TMScore1'].round(2)
    df['TMScore2'] = df['TMScore2'].round(2)
    
    df['RMSD1'] = df['RMSD1'].round(1)
    df['RMSD2'] = df['RMSD2'].round(1)
    return df

df = create_df(base_dir + sum1, base_dir + sum2, base_dir + casp)

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(8, 8))
# plt.ylim(0, 0.30)
# plt.xlim(0, 0.30)

In [ ]:
grid = sns.JointGrid(df.TMScore1, df.TMScore2, space=0, size=8, ratio=10)
grid.set_axis_labels("Model 1", "Model 2")
grid.plot_joint(plt.scatter, color="black")
plt.plot([0, 0.3], [0, 0.3], linewidth=1, linestyle='--')

In [ ]:
# tm_plot.savefig(base_dir + 'test_plot.eps', format='eps', dpi=1000)

In [ ]:
# fig = tm_plot.get_figure()
# fig.savefig(base_dir + 'test_plot.eps', format='eps', dpi=1000)

In [ ]:
grid.savefig(base_dir + 'test_plot.eps', format='eps', dpi=1000)

In [8]:
base_dir = 'input_output/'


In [9]:
import pandas as pd

In [10]:
df = pd.read_excel(base_dir + 'synlethdb.xlsx')

IndexError: list index out of range

In [ ]:
!pip install xlrd

In [14]:
f = open(base_dir + 'sophia.txt',"r+").read()

In [25]:
def str_to_int(x):
    try:
        return int(x)
    except ValueError:
        return x

In [26]:
list(map(str_to_int, list(set(f.split()))))

[24040035,
 21822123,
 27461052,
 27375368,
 19034564,
 18336191,
 21643007,
 29866747,
 23578175,
 16972754,
 30875717,
 29797327,
 24928852,
 21603316,
 29899826,
 16210249,
 22101337,
 22678161,
 25292178,
 29610289,
 25232030,
 19720857,
 15047861,
 20811722,
 12582246,
 26283727,
 30705406,
 24006065,
 18413751,
 27364904,
 20227038,
 25261849,
 16038621,
 19015239,
 24218601,
 25686104,
 22190494,
 20406170,
 'Pubmed_id',
 22010575,
 30333913,
 26603525,
 21321066,
 21144036,
 22955508,
 21814623,
 16326109,
 19878585,
 21706479,
 27958275,
 17568563,
 23419720,
 21978374,
 26938910,
 25139395,
 24927325,
 20568893,
 20376879,
 25103497,
 11481475,
 23204129,
 18794128,
 16177181,
 24809668,
 21737609,
 22073281,
 17005722,
 21980305,
 19525400,
 20615389,
 17121906,
 11504908,
 20700484,
 23948487,
 24317580,
 22120667,
 23536721,
 5564254,
 25558828,
 19847166,
 20860770,
 24425774,
 20858840,
 25171417,
 24377575,
 20191379,
 30266755,
 30323337,
 25955731,
 22412391,
 2177719